In [36]:
import importlib
import main
importlib.reload(main)
from main import *


## Preprocessing parameters

In [55]:
# The locaton of the EXAFS data
EXAFS_dir = "./Processed_EXAFS"
# The location of the FEFF calculations
FEFF_dir = "./FEFF"

gas_temp_regex = r"_(N2|CO|O2|He|H2)_(RT|\d+)"


assignment_dict = {
    'gas': 0,  # Assuming the first match is gas
    'temp': 1,       # The second match is the temp
}

pre_edge_kws: dict = {
    "e0": 11564,
    "step": None,
    "nnorm": 3,
    "nvict": 0,
    "pre1": -150,
    "pre2": -30,
    "norm1": 150,
    "norm2": 860,
}

autobk_kws: dict = {"rbkg": 1.2, "kmax": 16, "kweight": 2}

xftf_kws: dict = {"kmin": 2.5, "kmax": 12, "dk": 2, "window": "hanning"}


## Locate EXAFS data

In [41]:
all_exafs_data = glob.glob(EXAFS_dir + "/*.dat")
print(all_exafs_data)

['./Processed_EXAFS/11_CO_175.dat', './Processed_EXAFS/13_CO_225.dat', './Processed_EXAFS/3_H2_300_initial.dat', './Processed_EXAFS/4_He_RT_after_H2.dat', './Processed_EXAFS/10_CO_150.dat', './Processed_EXAFS/15_CO_275.dat', './Processed_EXAFS/14_CO_250.dat', './Processed_EXAFS/16_CO_300.dat', './Processed_EXAFS/7_CO_75.dat', './Processed_EXAFS/12_CO_200.dat', './Processed_EXAFS/1_O2_300.dat', './Processed_EXAFS/2_He_RT_after_O2.dat', './Processed_EXAFS/8_CO_100.dat', './Processed_EXAFS/9_CO_125.dat', './Processed_EXAFS/0_N2_RT_initial.dat', './Processed_EXAFS/6_CO_50.dat', './Processed_EXAFS/5_CO_RT.dat']


## Create Larch Groups

In [42]:
# 1 group
new_group = read_data(
    fname=all_exafs_data[0], 
    regex=gas_temp_regex, 
    skip=1, 
    assignment_dict=assignment_dict,
    pre_edge_kws= pre_edge_kws,
    autobk_kws= autobk_kws,
    xftf_kws= xftf_kws
)

In [56]:
# multiple groups
groups = read_multiple_spectra(
    all_exafs_data, 
    regex=gas_temp_regex, 
    skip=1, 
    assignment_dict=assignment_dict,
    pre_edge_kws= pre_edge_kws,
    autobk_kws= autobk_kws,
    xftf_kws= xftf_kws
)

## Fitting parameters

In [59]:
parm_dict = {
    "SO2": {"initial": 0.76, "vary": False, "global": True},
    "dele": {"initial": 0.0, "vary": True, "global": True},
    "ss2": {"initial": 0.003, "vary": True, "global": False},
    "N": {"initial": 11.0, "vary": True, "global": True},
    "delr": {"initial": 0.0, "vary": True, "global": False},
    "th": {"initial": 0.0, "vary": True, "global": False},
}

fit_range_param = {
    "kmin": 2.5,
    "kmax": 12,
    "kw": 3,
    "dk": 3,
    "window": "hanning",
    "rmin": 1.7,
    "rmax": 3.3,
}

In [61]:
# SiO2
dset, out, report, paths = run_fit(
    groups, parm_dict, FEFF_dir, 0, fit_range_param, write=True, save_name="./fitting_output/fit.out"
)

./FEFF
./FEFF/files.dat
Feffit Warning: unused variables: delr_100, delr_150, delr_175, delr_200, delr_225, delr_250, delr_275, delr_300, delr_50, delr_75, delr_RT, ss2_100, ss2_150, ss2_175, ss2_200, ss2_225, ss2_250, ss2_275, ss2_300, ss2_50, ss2_75, ss2_RT, th_100, th_150, th_175, th_200, th_225, th_250, th_275, th_300, th_50, th_75, th_RT
